### Importing Libraries

In [45]:
from dkube.sdk import *
import os

In [46]:
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kubernetes import client as k8s_client
from kfp import components
import json
from random import randint

### Initializing fields

In [47]:
user = os.getenv('USERNAME')
token = os.getenv('DKUBE_USER_ACCESS_TOKEN')
code_name = "sdk-pallavi"
dataset_name = "sdk-test-pallavi"
model_name = "sdk-pallavi"
framework = "tensorflow_1.14"
image = "ocdr/d3-datascience-tf-cpu:v1.14-3"
training_script = "python model.py"
transformer_script = "mnist/transformer.py"

### Load Components

In [48]:

dkube_training_op = components.load_component_from_file("/mnt/dkube/pipeline/components/training/component.yaml")
dkube_serving_op  = components.load_component_from_file("/mnt/dkube/pipeline/components/serving/component.yaml")


In [49]:

@dsl.pipeline(
  name='MNIST',
  description='A pipeline to train and serve the MNIST example.'
)
def mnist_pipeline(train_steps='200',
                   learning_rate='0.01',
                   batch_size='100',
                   token=token,code_name=code_name,dataset_name=dataset_name,model_name=model_name):
    
                   training = dkube_training_op(token, json.dumps({"image": image}),
                            framework="tensorflow", version="1.14",
                            program=code_name, run_script=training_script,
                            input_dataset_mounts='["tmp/data/"]',outputs='["sdk-pallavi"]',
                            output_mounts='["mnist/"]')
                   
                   serving  = dkube_serving_op(token, training.outputs['artifact'],
                            device="cpu", serving_image=json.dumps({"image": "ocdr/tensorflowserver:1.14"}),
                            transformer_image=json.dumps({"image": image}),
                            transformer_project=code_name,
                            transformer_code=transformer_script).after(training).set_display_name("MNIST Serving")

In [50]:
experiment_name = 'MNIST pl'
client = kfp.Client(existing_token=token)
mnist_experiment = client.create_experiment(name=experiment_name)

In [51]:
arguments = {"token":token}
compiler.Compiler().compile(mnist_pipeline, "mnist-pipeline.zip")
try:
    pipeline = client.upload_pipeline("mnist-pipeline.zip", pipeline_name = "mnist-pipeline")
except BaseException as e:
    print(e)
runid = 1


In [52]:
run = client.run_pipeline(mnist_experiment.id, job_name="[MNIST] Run" + str(runid), pipeline_id=pipeline.id, params=arguments)
runid += 1